Import packages

In [1]:
# import sys, os
# import pathlib
# # __location__ = os.getcwd()
# # __location__ = pathlib.Path(__location__)

# # sys.path.append('C:\\Users\\doore\\project\\snake_RL\\dmc\\domains')
# import snake
# import snake_v2
# import snake_v3


# from snake.envs.SnakeEnv import SnakeEnv # for ray env register
# from snake_v2.envs.SnakeEnv import SnakeEnv # for ray env register
from snake_v5.envs.SnakeEnv import SnakeEnv # for ray env register
import gymnasium as gym

# import rl 알고리즘
from ray.rllib.algorithms.ppo import PPOConfig 
from ray.rllib.algorithms.algorithm_config import AlgorithmConfig
from ray.rllib.policy.policy import Policy

from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

import numpy as np

Make Snake Env

In [2]:
env = gym.make('snake/SnakeEnv-v5', render_mode="human")

register_env("snake-v5", lambda config: SnakeEnv())


Load pre-learned policy

In [3]:
from ray.rllib.models import ModelCatalog
from ray.rllib.models.modelv2 import ModelV2
from ray.rllib.models.preprocessors import get_preprocessor
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.torch.recurrent_net import RecurrentNetwork as TorchRNN
from ray.rllib.models.tf.recurrent_net import RecurrentNetwork
from ray.rllib.utils.annotations import override
from ray.rllib.utils.framework import try_import_tf, try_import_torch

policy = Policy.from_checkpoint('C:/Users/doore/ray_results/PPO_snake-v5_2023-03-27_02-26-33khkire5h/checkpoint_000450')



In [23]:
## For Recurrent Policy

# lstm_dim = 64
# num_lstm_cells = 2
# initial_state = np.zeros((num_lstm_cells, lstm_dim))

_state = policy['default_policy'].get_initial_state()
_prev_action = np.zeros(14,)
_reward = 0
_obs, _ = env.reset()
_accum_reward = 0

for epi in range(3):
    for i in range(611):
        _act, _state, _ = policy['default_policy'].compute_single_action(obs=_obs.copy(), state=_state.copy(), prev_action=_prev_action.copy(), prev_reward=_reward)
        _prev_action = _act.copy()
        _obs, _reward, _, _, _dict = env.step(_act)
        _accum_reward = _accum_reward + _reward
        print(str(_accum_reward)+"  \r",end='')
        # print(_dict['forward_reward'])
    _obs, _ = env.reset()
    _accum_reward = 0
    print("\nepisode done...")
env.close()

-4.538207062794032    
episode done...
-18.359133346025587   
episode done...
3.894964233573446     
episode done...


In [ ]:
policy['default_policy'].model.get_parameter

Do simulation

In [ ]:
_obs, _ = env.reset()

c_reward = 0
for epi in range(10):
    for i in range(400):
        _act = policy['default_policy'].compute_single_action(_obs)
        _obs, _reward, _, _, _ = env.step(_act[0])
        c_reward = c_reward + _reward
    _obs, _ = env.reset()
    print(c_reward)
    c_reward = 0
    
env.close()

In [7]:
_state = policy['default_policy'].get_initial_state()
_state = np.tile(_state,(10,1))
# _state = np.concatenate(_state)

env.close()
# _obs, _ = env.reset()

_act, _state, _ = policy['default_policy'].compute_single_action(obs=_obs, state=_state)


2023-03-23 01:51:58,101	ERROR tf_run_builder.py:50 -- Error fetching: [<tf.Tensor 'var_scope_1/cond_1/Merge:0' shape=(?, 14) dtype=float32>, <tf.Tensor 'var_scope_1/Reshape_2:0' shape=(?, 32) dtype=float32>, {'action_prob': <tf.Tensor 'var_scope_1/Exp_1:0' shape=(?,) dtype=float32>, 'action_logp': <tf.Tensor 'var_scope_1/cond_2/Merge:0' shape=(?,) dtype=float32>, 'action_dist_inputs': <tf.Tensor 'var_scope_1/model_2/dense_6/BiasAdd:0' shape=(?, 28) dtype=float32>, 'vf_preds': <tf.Tensor 'var_scope_1/Reshape_3:0' shape=(?,) dtype=float32>}], feed_dict={<tf.Tensor 'var_scope_1/obs:0' shape=(?, 38) dtype=float32>: array([[ 0.00000000e+00, -1.00000000e+00, -1.00000000e+00,
        -1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -1.00000000e+00, -2.41623026e+00,
        -8.01598502e-01,  1.50000000e+01,  2.46497944e-02,
        -2.10706904e-02,  2

ValueError: Cannot feed value of shape (1, 32) for Tensor var_scope_1/state_in_0:0, which has shape (?, ?, 32)